In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 表格工作流程：TabNet管道

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/tabnet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/tabnet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabular_workflows/tabnet_on_vertex_pipelines.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

注意：该笔记本已在以下环境中进行了测试：

- Python版本 = 3.9

## 概述

本笔记本展示了如何使用Vertex AI表格工作流来运行TabNet算法。

了解有关[TabNet的表格工作流](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet)的更多信息。

### 目标

在本教程中，您将学习如何使用Vertex AI TabNet Tabular工作流在表格数据上创建分类模型。每个工作流都是[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)的托管实例。

本教程使用以下谷歌云ML服务和资源：

- Vertex AI Training
- Vertex AI Pipelines
- Cloud Storage

执行的步骤包括：

- 创建TabNet CustomJob。如果您知道要用于训练的超参数，则这是最佳选项。
- 创建TabNet HyperparameterTuningJob。这允许您获取适合您数据集的最佳超参数集。

培训完成后，每个流水线都将返回一个链接到Vertex Model UI。您可以使用UI部署模型，获取在线预测，或运行批处理预测。

### 数据集

您在此笔记本中使用的数据集是[银行营销](https://archive.ics.uci.edu/ml/datasets/bank+marketing)数据集。
该数据集包含与葡萄牙银行机构的直接营销活动（电话呼叫）相关的数据。此笔记本中二元分类任务的目标是预测客户是否订阅定期存款。

对于此笔记本，随机选择的原始数据集中约90%的行的子集已保存到`train.csv`文件中，并托管在Cloud Storage中。要下载文件，请单击[此处](https://storage.googleapis.com/cloud-samples-data-us-central1/vertex-ai/tabular-workflows/datasets/bank-marketing/train.csv)。

### 成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-pipeline-components

### 仅限于Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的 Google Cloud 项目

**无论您使用的是笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下API：Vertex AI API，云资源管理器 API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果您正在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。 了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需进行任何操作，因为您已经验证过身份。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参阅如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上授予云存储权限给您的服务帐户。

### 创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间工件。

当您使用云SDK提交训练作业时，您需要将包含训练代码的Python软件包上传到一个云存储桶中。Vertex AI会从这个包中运行代码。在本教程中，Vertex AI还会将训练作业产生的训练模型保存在同一个存储桶中。使用这个模型工件，然后您可以创建Vertex AI模型资源，并用于预测。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 关于服务账户和权限的注意事项

**默认情况下不需要任何配置**，如果遇到任何与权限相关的问题，请确保服务账户具有[TabNet、Wide & Deep、Prophet文档](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/service-accounts#fte-workflow)中列出的所需角色。

服务账户

您可以使用服务账户来创建 Vertex AI Pipeline 作业。如果您不想使用项目的 Compute Engine 服务账户，将 `SERVICE_ACCOUNT` 设置为其他服务账户 ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务账户访问权限
运行以下命令，为您的服务账户授予读取和写入管道 artifacts 在您在上一步创建的存储桶中的访问权限。您只需要对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
# Import required modules
import os
from typing import Any, Dict, List

from google.cloud import aiplatform, storage
from google_cloud_pipeline_components.preview.automl.tabular import \
    utils as automl_tabular_utils

### 初始化 Vertex AI SDK 用于 Python

为您的项目初始化 Vertex AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## 定义辅助函数
定义以下辅助函数：

- `get_model_artifacts_path`：从任务详情中获取模型文件路径。
- `get_model_uri`：从任务详情中获取模型URI。
- `get_bucket_name_and_path`：获取存储桶名称和路径。
- `download_from_gcs`：从存储桶下载内容。
- `write_to_gcs`：将内容上传到存储桶。
- `get_task_detail`：通过任务名称获取任务详情。
- `get_model_name`：从管道作业ID中获取模型名称。
- `get_evaluation_metrics`：从管道任务详情中获取评估指标。

In [ ]:
# Get the model artifacts path from task details.


def get_model_artifacts_path(task_details: List[Dict[str, Any]], task_name: str) -> str:
    task = get_task_detail(task_details, task_name)
    return task.outputs["unmanaged_container_model"].artifacts[0].uri


# Get the model uri from the task details.
def get_model_uri(task_details: List[Dict[str, Any]]) -> str:
    task = get_task_detail(task_details, "model-upload")
    # in format https://<location>-aiplatform.googleapis.com/v1/projects/<project_number>/locations/<location>/models/<model_id>
    model_id = task.outputs["model"].artifacts[0].uri.split("/")[-1]
    return f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model_id}?project={PROJECT_ID}"


# Get the bucket name and path.
def get_bucket_name_and_path(uri: str) -> str:
    no_prefix_uri = uri[len("gs://") :]
    splits = no_prefix_uri.split("/")
    return splits[0], "/".join(splits[1:])


# Get the content from the bucket.
def download_from_gcs(uri: str) -> str:
    bucket_name, path = get_bucket_name_and_path(uri)
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    return blob.download_as_string()


# Upload content into the bucket.
def write_to_gcs(uri: str, content: str):
    bucket_name, path = get_bucket_name_and_path(uri)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    blob.upload_from_string(content)


# Get the task details by using task name.
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail


# Get the model name from pipeline job ID.
def get_model_name(job_id: str) -> str:
    pipeline_task_details = aiplatform.PipelineJob.get(
        job_id
    ).gca_resource.job_detail.task_details
    upload_task_details = get_task_detail(pipeline_task_details, "model-upload")
    return upload_task_details.outputs["model"].artifacts[0].metadata["resourceName"]


# Get the evaluation metrics.
def get_evaluation_metrics(
    task_details: List[Dict[str, Any]],
) -> str:
    ensemble_task = get_task_detail(task_details, "model-evaluation")
    return download_from_gcs(
        ensemble_task.outputs["evaluation_metrics"].artifacts[0].uri
    )

## 定义培训规范

在创建培训作业之前，在本节中您需要创建以下步骤：

1. 配置源数据集。
2. 配置特征转换过程。
3. 配置特征选择过程。
4. 设置运行培训过程所需的参数。

### 配置数据集

您可以定义以下参数之一：

- `data_source_csv_filenames`：CSV数据源。您需要指定`train.csv`文件在数据集部分描述的云存储路径。
- `data_source_bigquery_table_path`：BigQuery数据源。由于使用了云存储源，因此将此设置为无。

***注意***：请注意数据集的位置必须与用于启动培训管道的服务位置（即 `REGION`）相同。

In [ ]:
data_source_csv_filenames = "gs://cloud-samples-data-us-central1/vertex-ai/tabular-workflows/datasets/bank-marketing/train.csv"
data_source_bigquery_table_path = (
    None  # @param {type:"string"}, format: bq://bq_project.bq_dataset.bq_table
)

### 配置特征转换

可以使用Feature Transform Engine（FTE）特定的配置来指定转换。FTE支持基于TensorFlow的行级和基于BigQuery的数据集级转换。

* **基于TensorFlow的行级转换**：
  * 完全自动转换：FTE根据数据统计信息自动为每个输入列配置一组内置转换。这可以通过训练管线中的`tf_auto_transform_features`进行设置。
  * 完全指定的转换：对输入列的所有转换都明确指定为FTE的内置转换。还支持对单个列进行多个转换的串联。这些转换可以保存到JSON配置文件中，并通过训练管线的`tf_transformations_path`参数进行指定。
  * 自定义转换：自定义的、自定义的转换功能，您可以定义和导入自己的转换函数，并与其他FTE的内置转换一起使用。您可以将自定义转换指定为JSON对象数组，并通过训练管线的`tf_custom_transformation_definitions`参数传递。

* **基于BigQuery的数据集级转换**：
  * 完全指定的转换：对输入列的所有转换都明确指定为FTE的内置转换。这些转换可以通过训练管线的`dataset_level_transformations`参数作为JSON对象数组进行指定。
  * 自定义转换：自定义的、自定义的转换功能，您可以定义和导入自己的转换函数，并与其他FTE的内置转换一起使用。您可以将自定义转换指定为JSON对象数组，并通过训练管线的`dataset_level_custom_transformation_definitions`参数进行传递。

在下面，通过将要传递给训练管线的`tf_auto_transform_features`参数指定为输入特征列表，来配置完全自动转换。

了解更多关于[特征转换配置](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.31/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.FeatureTransformEngineOp)。

In [ ]:
auto_transform_features = [
    "age",
    "job",
    "marital",
    "education",
    "default",
    "balance",
    "housing",
    "loan",
    "contact",
    "day",
    "month",
    "duration",
    "campaign",
    "pdays",
    "previous",
    "poutcome",
]

### 配置特征选择

除了转换之外，您还可以通过特征转换引擎应用特征选择，仅使用由支持的算法评估的高排名特征。如果启用，它将在数据集级别转换后立即应用，并排除任何未被选定的特征。

要启用它，您需要将`run_feature_selection`设置为True。

要配置要使用的算法和要选择的特征数量，您需要配置`feature_selection_algorithm`和`max_selected_features`参数。

了解更多关于[特征选择算法和配置](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.31/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.FeatureTransformEngineOp)。

In [ ]:
RUN_FEATURE_SELECTION = True  # @param {type:"boolean"}

FEATURE_SELECTION_ALGORITHM = "AMI"  # @param {type:"string"}

MAX_SELECTED_FEATURES = 10  # @param {type:"integer"}

设置训练配置

现在，您可以定义以下培训参数：

- `target_column`：目标列名称。
- `prediction_type`：模型将产生的预测类型。'classification'或'regression'。
- `predefined_split_key`：预定义分割列名称。
- `timestamp_split_key`：时间戳分割列名称。
- `stratified_split_key`：分层分割列名称。
- `training_fraction`：培训分数。
- `validation_fraction`：验证分数。
- `test_fraction`：测试分数。
- `weight_column`：权重列名称。
- `run_evaluation`：是否在培训过程中运行评估步骤。

In [ ]:
run_evaluation = True  # @param {type:"boolean"}
prediction_type = "classification"
target_column = "deposit"

# Fraction split
training_fraction = 0.8
validation_fraction = 0.1
test_fraction = 0.1

timestamp_split_key = None  # timestamp column name when using timestamp split
stratified_split_key = None  # target column name when using stratified split

predefined_split_key = None
if predefined_split_key:
    training_fraction = None
    validation_fraction = None
    test_fraction = None

weight_column = None

## 为 Dataflow 设置 VPC 配置

在本节中，您需要定义以下参数：

- `dataflow_subnetwork`：Dataflow 的完全限定子网络名称，当为空时将使用默认子网络。请参阅[示例](https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications)。
- `dataflow_use_public_ips`：指定 Dataflow 工作节点是否使用公共 IP 地址。

如果需要使用自定义的 Dataflow 子网络，您可以通过`dataflow_subnetwork` 参数进行设置。要求是：
1. `dataflow_subnetwork` 必须是完全限定的子网络名称。
   [[参考资料](https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications)]
1. 以下服务账号必须在指定的 Dataflow 子网络上分配[计算网络用户角色](https://cloud.google.com/compute/docs/access/iam#compute.networkUser) ：
    1. Compute Engine 默认服务账号：PROJECT_NUMBER-compute@developer.gserviceaccount.com
    1. Dataflow 服务账号：service-PROJECT_NUMBER@dataflow-service-producer-prod.iam.gserviceaccount.com

如果您的项目启用了 VPC-SC，请确保以下事项：

1. 用于 VPC-SC 的 Dataflow 子网络已经为 Dataflow 正确配置。
   参见[参考文档](https://cloud.google.com/dataflow/docs/guides/routes-firewall)。
1. `dataflow_use_public_ips` 已设置为 False。

In [ ]:
dataflow_subnetwork = ""  # @param {type:"string"}
dataflow_use_public_ips = True  # @param {type:"boolean"}

定制TabNet CustomJob配置并创建管道

如果您确切地知道要在模型训练中使用哪些超参数值，创建TabNet CustomJob是最佳选择。它比HyperparameterTuningJob使用更少的训练资源。

在下面的示例中，您可以配置以下关键参数：

- `root_dir`：管道组件的根GCS目录。
- `worker_pool_specs_override`：用于覆盖训练和评估工作人员池规范的字典。该字典应遵循特定格式。TabNet支持使用CPU和GPU进行训练。
- `learning_rate`：线性优化器使用的学习率。
- `max_steps`：训练器运行的步数。
- `max_train_secs`：训练器运行的秒数。

了解有关[管道输入和模型超参数](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_tabnet_trainer_pipeline_and_parameters)的更多信息。

了解有关[创建管道作业所需的参数](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run)的更多信息。

In [ ]:
# set a unique display name for your pipeline
pipeline_job_id = "tabnet-unique"  # @param {type: "string"}
# set the root dir
pipeline_job_root_dir = os.path.join(BUCKET_URI, "tabnet_custom_job")
# set the worker pool specs
worker_pool_specs_override = [
    {"machine_spec": {"machine_type": "c2-standard-16"}}  # Override for TF chief node
]
# set the learning rate
learning_rate = 0.01
# max_steps and/or max_train_secs must be set. If both are
# specified, training stop after either condition is met.
# By default, max_train_secs is set to -1.
max_steps = 20

max_train_secs = -1

# To test GPU training, the worker_pool_specs_override can be specified like this.
# worker_pool_specs_override =  [
#     {"machine_spec": {
#       'machine_type': "n1-highmem-32",
#       "accelerator_type": "NVIDIA_TESLA_V100",
#       "accelerator_count": 2
#       }
#     }
#   ]

# define the pipeline
# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
(
    template_path,
    parameter_values,
) = automl_tabular_utils.get_tabnet_trainer_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=pipeline_job_root_dir,
    max_steps=max_steps,
    max_train_secs=max_train_secs,
    learning_rate=learning_rate,
    target_column=target_column,
    prediction_type=prediction_type,
    tf_auto_transform_features=auto_transform_features,
    run_feature_selection=RUN_FEATURE_SELECTION,
    feature_selection_algorithm=FEATURE_SELECTION_ALGORITHM,
    max_selected_features=MAX_SELECTED_FEATURES,
    training_fraction=training_fraction,
    validation_fraction=validation_fraction,
    test_fraction=test_fraction,
    data_source_csv_filenames=data_source_csv_filenames,
    data_source_bigquery_table_path=data_source_bigquery_table_path,
    worker_pool_specs_override=worker_pool_specs_override,
    dataflow_use_public_ips=dataflow_use_public_ips,
    dataflow_subnetwork=dataflow_subnetwork,
    run_evaluation=run_evaluation,
)

# create the pipeline job
training_pipeline_job = aiplatform.PipelineJob(
    display_name=pipeline_job_id,
    template_path=template_path,
    job_id=pipeline_job_id,
    pipeline_root=pipeline_job_root_dir,
    parameter_values=parameter_values,
    enable_caching=False,
)

# run the pipeline
training_pipeline_job.run(service_account=SERVICE_ACCOUNT)

### 前往顶点模型界面

通过下面单元格生成的链接，您可以部署模型并进行在线预测或批量预测。

In [ ]:
tabnet_trainer_pipeline_task_details = aiplatform.PipelineJob.get(
    pipeline_job_id
).gca_resource.job_detail.task_details
CUSTOM_JOB_MODEL = get_model_name(pipeline_job_id)
print("model uri:", get_model_uri(tabnet_trainer_pipeline_task_details))
print(
    "model artifacts:",
    get_model_artifacts_path(tabnet_trainer_pipeline_task_details, "tabnet-trainer"),
)

## 自定义 TabNet 超参数调优作业配置并创建流水线

为了在您的数据集上获得最佳的超参数设置，建议运行一个超参数调优作业。

可以调优的超参数通过可选的`study_spec_parameters_override`参数设置。您提供一个名为`get_tabnet_study_spec_parameters_override`的辅助函数以获取这些超参数。在这个辅助函数中，您提供：

- `dataset_size_bucket`：'small'（< 1M行），'medium'（1M - 100M行）或'large'（> 100M行）之一。
- `training_budget_bucket`：'small'（< \\$600），'medium'（\\$600 - \\$2400）或'large'（> \\$2400）之一。
- `prediction_type`：模型要生成的预测类型。“classification”或“regression”。

然后，您获得超参数和范围的列表。`study_spec_parameters_override`可以为空，或者可以指定一个或多个上述超参数。对于未指定的超参数，您可以在流水线中设置它们的范围。了解有关[可用于调优的超参数](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_tabnet_trainer_pipeline_and_parameters)的更多信息。

除了超参数外，超参数调优作业需要以下值：

- `root_dir`：流水线组件的根 GCS 目录。
- `worker_pool_specs_override`：用于覆盖训练和评估工作池规格的字典。字典应遵循特定格式。TabNet 支持使用 CPU 和 GPU 进行训练。
- `study_spec_metric_id`：要优化的度量标准，可能的值为['loss', 'average_loss', 'rmse', 'mae', 'mql', 'accuracy', 'auc', 'precision', 'recall']。
- `study_spec_metric_goal`：度量标准的优化目标，可能的值为"MAXIMIZE"，"MINIMIZE"。
- `max_trial_count`：期望的总试验次数。
- `parallel_trial_count`：并行运行的试验次数。
- `max_failed_trial_count`：需要在失败之前看到的失败试验次数。如果设置为0，则 Vertex AI 决定整个作业失败前必须有多少次试验失败。
- `study_spec_algorithm`：用于研究的搜索算法。其中之一为'ALGORITHM_UNSPECIFIED'，'GRID_SEARCH'或'RANDOM_SEARCH'。

了解有关[超参数调优作业参数](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.23/google_cloud_pipeline_components.experimental.automl.tabular.html#google_cloud_pipeline_components.experimental.automl.tabular.utils.get_tabnet_hyperparameter_tuning_job_pipeline_and_parameters)的更多信息。

可以配置多个试验。根据`study_spec_metrics`中指定的度量标准，流水线返回最佳试验。在下面的示例中，您返回具有最低损失值的试验。

In [ ]:
# set a unique display name for pipeline
pipeline_job_id = "tabnet-hpt-unique"  # @param {type: "string"}
# set the root dir
pipeline_job_root_dir = os.path.join(BUCKET_URI, "tabnet_hyperparameter_tuning_job")
# set the worker pool specs
worker_pool_specs_override = [
    {"machine_spec": {"machine_type": "c2-standard-16"}}  # Override for TF chief node
]
# set the metric
study_spec_metric_id = "loss"
# set the objective for metric
study_spec_metric_goal = "MINIMIZE"

# To test GPU training, the worker_pool_specs_override can be specified like this.
# worker_pool_specs_override =  [
#    {
#       "machine_spec":{
#          "machine_type":"n1-highmem-32",
#          "accelerator_type":"NVIDIA_TESLA_V100",
#          "accelerator_count":2
#       }
#    }
# ]


# define the component to get the hyperparameters
# max_steps and/or max_train_secs must be set. If both are
# specified, training stop after either condition is met.
# By default, max_train_secs is set to -1 and max_steps is set to
# an appropriate range given dataset_size and training budget.
study_spec_parameters_override = (
    automl_tabular_utils.get_tabnet_study_spec_parameters_override(
        dataset_size_bucket="small",
        prediction_type=prediction_type,
        training_budget_bucket="small",
    )
)

# define the hyperparameter tuning pipeline
# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
(
    template_path,
    parameter_values,
) = automl_tabular_utils.get_tabnet_hyperparameter_tuning_job_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=pipeline_job_root_dir,
    target_column=target_column,
    prediction_type=prediction_type,
    tf_auto_transform_features=auto_transform_features,
    run_feature_selection=RUN_FEATURE_SELECTION,
    feature_selection_algorithm=FEATURE_SELECTION_ALGORITHM,
    max_selected_features=MAX_SELECTED_FEATURES,
    training_fraction=training_fraction,
    validation_fraction=validation_fraction,
    test_fraction=test_fraction,
    data_source_csv_filenames=data_source_csv_filenames,
    data_source_bigquery_table_path=data_source_bigquery_table_path,
    study_spec_metric_id=study_spec_metric_id,
    study_spec_metric_goal=study_spec_metric_goal,
    study_spec_parameters_override=study_spec_parameters_override,
    max_trial_count=1,
    parallel_trial_count=1,
    max_failed_trial_count=0,
    worker_pool_specs_override=worker_pool_specs_override,
    dataflow_use_public_ips=dataflow_use_public_ips,
    dataflow_subnetwork=dataflow_subnetwork,
    run_evaluation=True,
)

# create the pipeline job
tuning_pipeline_job = aiplatform.PipelineJob(
    display_name=pipeline_job_id,
    template_path=template_path,
    job_id=pipeline_job_id,
    pipeline_root=pipeline_job_root_dir,
    parameter_values=parameter_values,
    enable_caching=False,
)

# run the pipeline job
tuning_pipeline_job.run(service_account=SERVICE_ACCOUNT)

### 前往顶点模型UI

通过下面单元格生成的链接，您可以部署模型并进行在线预测或批量预测。

In [ ]:
tabnet_hpt_pipeline_task_details = aiplatform.PipelineJob.get(
    pipeline_job_id
).gca_resource.job_detail.task_details
HPT_JOB_MODEL = get_model_name(pipeline_job_id)

print("model uri:", get_model_uri(tabnet_hpt_pipeline_task_details))
print(
    "model artifacts:",
    get_model_artifacts_path(
        tabnet_hpt_pipeline_task_details, "get-best-hyperparameter-tuning-job-trial"
    ),
)

清理顶点和BigQuery资源

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 从CustomJob管道中的管道
- 从HyperparameterTuningJob管道中的管道
- 从CustomJob管道中的模型
- 从HyperparameterTuningJob管道中的模型
- Cloud Storage存储桶（将`delete_bucket`设置为True可删除存储桶）

In [ ]:
# Delete the training pipeline job
training_pipeline_job.delete()

# Delete the tuning pipeline job
tuning_pipeline_job.delete()

# Delete model resources
custom_job_model = aiplatform.Model(CUSTOM_JOB_MODEL)
hpt_job_model = aiplatform.Model(HPT_JOB_MODEL)
custom_job_model.delete()
hpt_job_model.delete()

# Delete bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI